<div style="width:100%; overflow:hidden; background-color:#F1F1E6; padding: 10px; border-style: outset; color:#17469e">
    <div style="width: 80%; float: left;">
    <h2 align="center">Universidad de Sonora</h2>
    <hr style="border-width: 3px; border-color:#17469e">
          <h1>Reconocimiento de patrones: Preparación de los datos</h1>          
          <h4>Ramón Soto C. <a href="mailto:rsotoc@moviquest.com/">(rsotoc@moviquest.com)</a></h4>
    </div>
    <div style="float: right;">
    <img src="images/escudo_unison.png">
    </div>
</div>

## Caso de estudio: [*Stack Overflow 2018 Developer Survey*](https://www.kaggle.com/stackoverflow/stack-overflow-2018-developer-survey)

Como caso de estudio principal en el presente curso hemos seleccionado la encuesta de desarrolladores 2018 de *Stack Overflow* disponible en [Kaggle](https://www.kaggle.com). En este esta etapa realizaremos el análisis de agrupamientos.

### 4. Modelado

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7; ">
Los datos de nuestro caso de estudio, después de las tareas de preparación de los datos, son los siguientes:
</div>

In [1]:
"""
Reconocimiento de patrones: Preparación de los datos
"""

import pandas as pd
import numpy as np
import json
import pickle

from collections import Counter
from operator import itemgetter
from IPython.display import display, HTML

pd.set_option('display.max_columns', 130)
pd.set_option('max_colwidth', 80)

In [5]:
path = "Data sets/Stack Overflow Survey/"

# Recuperar encabezados de columnas en orden original
with open(path + 'survey_results_public_transformed.headers', 'rb') as file:  
    headers = pickle.load(file)

# Recuperar diccionarios... sólo por si se requieren
with open(path + 'survey_results_public_transformed.dicts', 'rb') as file:  
    dict_of_dicts = pickle.load(file)

with open(path + 'survey_results_public_transformed.json') as f:
    dict_json = json.load(f)
df = pd.DataFrame.from_dict(dict_json)

# Reordenar las columnas de acuerdo al orden original
df = df.reindex_axis(headers, axis=1)

print(df.info(), "\n")

/Users/rsotoc/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98443 entries, 0 to 98442
Data columns (total 85 columns):
Hobby                          98443 non-null int64
OpenSource                     98443 non-null int64
Country                        98443 non-null object
Student                        98443 non-null int64
Employment                     98443 non-null int64
FormalEducation                98443 non-null int64
UndergradMajor                 90343 non-null float64
CompanySize                    98443 non-null int64
DevType                        98443 non-null object
YearsCoding                    98443 non-null int64
YearsCodingProf                98443 non-null int64
JobSatisfaction                98443 non-null int64
CareerSatisfaction             98443 non-null int64
HopeFiveYears                  75718 non-null float64
JobSearchStatus                98443 non-null int64
LastNewJob                     84581 non-null float64
UpdateCV                       65539 non-null float

In [6]:
display(df.head(10))

,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,UpdateCV,ConvertedSalary,CommunicationTools,TimeFullyProductive,EducationTypes,SelfTaughtTypes,HackathonReasons,AgreeDisagree1,AgreeDisagree2,AgreeDisagree3,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,FrameworkWorkedWith,FrameworkDesireNextYear,IDE,OperatingSystem,NumberMonitors,Methodology,VersionControl,CheckInCode,AdBlocker,AdBlockerDisable,AdBlockerReasons,AdsAgreeDisagree1,AdsAgreeDisagree2,AdsAgreeDisagree3,AdsActions,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,EducationParents,RaceEthnicity,Age,Dependents,SurveyTooLong,SurveyEasy,LanguageWorkedWith,AssessJob,AssessBenefits,JobContactPriorities,JobEmailPriorities,AdsPriorities
0,1,0,KEN,0,1,1,10.0,4,[12],7,7,1,1,2.0,2,3.0,7.0,0.0,[8],3.0,"[3, 8]","[0, 3, 5, 7]",[1],3.0,3.0,2.0,"[5, 14, 18, 27]","[1, 12, 14, 17, 18, 19]","[1, 12, 14, 17, 18, 19]","[0, 5, 8, 14]","[0, 5, 8, 14]","[3, 6]","[3, 6]","[7, 17, 19]",1.0,1,"[0, 9]",[1],2.0,2.0,2.0,[1],3.0,3.0,3.0,"[2, 3]",1.0,0.0,3.0,1.0,1.0,2.0,2.0,2.0,10,5.0,2.0,3.0,1.0,3.0,5,2.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,0.0,3.0,[2],1.0,1.0,[0],1.0,1.0,0.0,4.0,"[5, 14, 18, 27]","[10.0, 7.0, 8.0, 1.0, 2.0, 5.0, 3.0, 4.0, 9.0, 6.0]",[],"[3.0, 1.0, 4.0, 2.0, 5.0]","[5.0, 6.0, 7.0, 2.0, 1.0, 4.0, 3.0]","[1.0, 5.0, 4.0, 7.0, 2.0, 6.0, 3.0]"
1,1,1,GBR,0,0,1,3.0,2,"[4, 7, 12, 20]",8,3,2,4,6.0,0,4.0,6.0,70841.0,"[0, 5, 7, 8]",3.0,"[1, 8]","[5, 7]",[],0.0,0.0,2.0,"[12, 27]","[11, 17, 18]",[17],[14],[14],[3],[6],"[5, 15, 17]",1.0,2,[],"[1, 4]",0.0,2.0,3.0,[5],1.0,0.0,0.0,[],3.0,3.0,3.0,1.0,0.0,0.0,2.0,2.0,10,0.0,2.0,0.0,2.0,0.0,7,2.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,2.0,3.0,[0],2.0,1.0,[6],2.0,1.0,0.0,2.0,"[1, 18, 27]","[1.0, 7.0, 10.0, 8.0, 2.0, 5.0, 4.0, 3.0, 6.0, 9.0]","[1.0, 5.0, 3.0, 7.0, 10.0, 4.0, 11.0, 9.0, 6.0, 2.0, 8.0]","[3.0, 1.0, 5.0, 2.0, 4.0]","[1.0, 3.0, 4.0, 5.0, 2.0, 6.0, 7.0]","[3.0, 5.0, 1.0, 4.0, 6.0, 7.0, 2.0]"
2,1,1,USA,0,0,0,6.0,4,"[10, 12]",5,9,3,3,2.0,2,3.0,NaN,0.0,[],NaN,[],[],[],NaN,NaN,NaN,[],[],[],[],[],[],[],[],NaN,None,[],[],NaN,NaN,NaN,[],NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],NaN,NaN,NaN,NaN,[],[],[],[],[],[]
3,0,0,USA,0,0,1,6.0,3,[12],3,1,4,6,2.0,2,3.0,1.0,NaN,[],5.0,"[0, 8]","[0, 5, 7, 8]",[],1.0,1.0,4.0,"[1, 3, 5, 14, 18, 31, 34]","[12, 19]","[12, 19]",[5],[5],[],"[0, 1, 6]","[18, 19]",3.0,2,"[0, 4, 9]",[1],2.0,2.0,3.0,[4],0.0,1.0,1.0,[3],1.0,1.0,0.0,0.0,1.0,3.0,2.0,2.0,10,1.0,2.0,0.0,2.0,0.0,8,2.0,3.0,3.0,3.0,3.0,3.0,5.0,2.0,3.0,1.0,[],3.0,7.0,[6],2.0,0.0,0.0,2.0,"[1, 3, 5, 14, 18, 31, 34]",[],[],[],[],[]
4,1,0,ZAF,3,0,8,6.0,2,"[2, 6, 13, 18, 19]",9,0,7,3,6.0,2,0.0,7.0,21426.0,[5],5.0,"[5, 6, 8]","[0, 5, 7]",[],3.0,0.0,4.0,"[0, 1, 2, 4, 22, 31]","[9, 16, 17, 19]","[9, 16, 17]","[4, 22]","[4, 22]",[],[],"[10, 18, 19]",3.0,2,"[1, 3]",[6],5.0,1.0,0.0,[0],1.0,1.0,2.0,"[0, 2]",0.0,0.0,3.0,1.0,1.0,3.0,2.0,2.0,10,2.0,2.0,4.0,1.0,1.0,5,2.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,0.0,3.0,[],1.0,7.0,[6],0.0,1.0,0.0,2.0,"[1, 2, 4, 17, 22, 28, 31]","[8.0, 5.0, 7.0, 1.0, 2.0, 6.0, 4.0, 3.0, 10.0, 9.0]","[1.0, 10.0, 2.0, 4.0, 8.0, 3.0, 11.0, 7.0, 5.0, 9.0, 6.0]","[2.0, 1.0, 4.0, 5.0, 3.0]","[7.0, 3.0, 6.0, 2.0, 1.0, 4.0, 5.0]","[2.0, 3.0, 4.0, 6.0, 1.0, 7.0, 5.0]"
5,1,0,GBR,0,0,1,6.0,1,"[0, 4, 11, 12]",9,7,3,7,6.0,0,1.0,2.0,41671.0,"[0, 4, 5, 6]",0.0,"[4, 5, 8]","[3, 7]","[0, 4, 5]"

<div style="margin-top: 6px; border: 1px solid #cfcfcf; padding: 8px 12px; border-radius:2px; background-color:#f7f7f7;">
En esta etapa utilizaremos diversas técnicas de modelado, para analizar los datos de estudio. El primer problema a resolver será el contar con funciones adecuadas para evaluar la distancia entre vectores:
</div>

In [3]:
def get_values(col_name):
    full_list = ";".join(col_name)
    each_word = full_list.split(";")
    each_word = sorted(list(Counter(each_word).keys()))
    return each_word